In [3]:
import cvxpy as cp

vertices = ["s", "r", "c", "b", "d"]
edges = {
    ("s", "r"): 3,
    ("s", "c"): 5,
    ("r", "c"): 1,
    ("r", "b"): 7,
    ("r", "d"): 4,
    ("c", "d"): 2,
    ("b", "d"): 3,
}

source = "s"
destination = "d"

# Decision variables: flow on each edge (non-negative)
edge_vars = {edge: cp.Variable(name=f"x_{edge}", nonneg=True) for edge in edges}

# Objective function: minimize total cost of the flow
total_cost = cp.sum([edges[edge] * edge_vars[edge] for edge in edges])

# Supply and demand at each node
b_v = {v: 0 for v in vertices}
b_v[source] = 1   # Supply at source node
b_v[destination] = -1  # Demand at destination node

# Constraints: flow conservation at each node
constraints = []
for v in vertices:
    inflow_edges = [e for e in edges if e[1] == v]
    outflow_edges = [e for e in edges if e[0] == v]
    inflow = cp.sum([edge_vars[e] for e in inflow_edges]) if inflow_edges else 0
    outflow = cp.sum([edge_vars[e] for e in outflow_edges]) if outflow_edges else 0
    constraints.append(outflow - inflow == b_v[v])

# Formulate and solve the LP problem
prob = cp.Problem(cp.Minimize(total_cost), constraints)
prob.solve()

# Output the results
print(f"Shortest path cost from {source} to {destination}: {prob.value}")
print("Edges in the shortest path:")
for edge in edges:
    flow = edge_vars[edge].value
    if flow > 1e-5:  # Consider flows greater than a small threshold
        print(f"{edge} with flow {flow}")



Shortest path cost from s to d: 6.00000002892718
Edges in the shortest path:
('s', 'r') with flow 0.9999999839979391
('r', 'c') with flow 0.9999999864918978
('c', 'd') with flow 1.000000002493966
